# Segmentation d'images — Baseline DeepLabV3+ avec MLflow

Notebook baseline pour entraîner un modèle de segmentation sémantique avec :
- Dataset images/masques alignés
- DeepLabV3+ (ResNet-50)
- Boucles train/val propres
- **MLflow** (paramètres, métriques, artefacts, modèle)
- Visualisations: masques colorisés et overlays
- Métriques: mIoU et IoU par classe (simple)

> Les commentaires sont sobres et placés par blocs logiques.

## Imports et configuration

In [ ]:
# Imports principaux
import os
import time
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
from torchvision.transforms.functional import to_tensor
from torchvision.models.segmentation import deeplabv3_resnet50
from PIL import Image
from tqdm import tqdm

# MLflow
import mlflow
import mlflow.pytorch

# Affichage
import matplotlib.pyplot as plt
from pathlib import Path

device = "cuda" if torch.cuda.is_available() else "cpu"
print("device:", device)

## Configuration MLflow

In [ ]:
# Répertoire local MLflow (modifiable : serveur distant possible)
mlflow.set_tracking_uri("mlruns")
mlflow.set_experiment("segmentation_baseline_deeplabv3")

# On garde un logging manuel et explicite
mlflow.autolog(disable=True)

## Utilitaires: palette et visualisation

In [ ]:
# Palette simple id->couleur (à adapter à votre ontologie)
# Ici on prévoit jusqu'à 20 classes. Modifiez selon vos besoins.
DEFAULT_PALETTE = [
    (0, 0, 0),       # 0: background
    (128, 64, 128),  # 1: road
    (244, 35, 232),  # 2: sidewalk
    (70, 70, 70),    # 3: building
    (102, 102, 156), # 4: wall
    (190, 153, 153), # 5: fence
    (153, 153, 153), # 6: pole
    (250, 170, 30),  # 7: traffic light
    (220, 220, 0),   # 8: traffic sign
    (107, 142, 35),  # 9: vegetation
    (152, 251, 152), # 10: terrain
    (70, 130, 180),  # 11: sky
    (220, 20, 60),   # 12: person
    (255, 0, 0),     # 13: rider
    (0, 0, 142),     # 14: car
    (0, 0, 70),      # 15: truck
    (0, 60, 100),    # 16: bus
    (0, 80, 100),    # 17: train
    (0, 0, 230),     # 18: motorcycle
    (119, 11, 32),   # 19: bicycle
]

def colorize_mask(mask, palette=DEFAULT_PALETTE):
    # mask: (H, W) tensor/ndarray d'indices de classe
    mask_np = mask if isinstance(mask, np.ndarray) else mask.cpu().numpy()
    h, w = mask_np.shape
    out = np.zeros((h, w, 3), dtype=np.uint8)
    n = min(len(palette), mask_np.max() + 1 if mask_np.size else len(palette))
    for idx in range(n):
        out[mask_np == idx] = palette[idx % len(palette)]
    return out

def overlay(image, color_mask, alpha=0.5):
    # image: torch Tensor [C,H,W] ou ndarray [H,W,3] en 0..1 ; color_mask: uint8
    if isinstance(image, torch.Tensor):
        img = image.permute(1,2,0).cpu().numpy()
    else:
        img = image
    if img.max() <= 1.0:
        base = (img * 255).astype(np.uint8)
    else:
        base = img.astype(np.uint8)
    return (alpha * color_mask + (1 - alpha) * base).astype(np.uint8)

def show_triplet(img_t, mask_pred, title=""):
    # Affiche image, masque colorisé, overlay
    cmask = colorize_mask(mask_pred)
    ov = overlay(img_t, cmask, alpha=0.5)
    plt.figure(figsize=(12,4))
    plt.subplot(1,3,1); plt.imshow(img_t.permute(1,2,0)); plt.title("Image"); plt.axis("off")
    plt.subplot(1,3,2); plt.imshow(cmask); plt.title("Masque (couleurs)"); plt.axis("off")
    plt.subplot(1,3,3); plt.imshow(ov); plt.title("Overlay"); plt.axis("off")
    if title:
        plt.suptitle(title)
    plt.tight_layout()

## Dataset et Dataloaders

In [ ]:
class SegmentationDataset(Dataset):
    # Dataset images + masques alignés (mêmes noms de fichiers)
    def __init__(self, img_dir, mask_dir, transform=None):
        self.img_dir = img_dir
        self.mask_dir = mask_dir
        self.transform = transform
        self.images = sorted([f for f in os.listdir(img_dir) if not f.startswith(".")])

    def __len__(self):
        return len(self.images)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.images[idx])
        mask_path = os.path.join(self.mask_dir, self.images[idx])
        img = Image.open(img_path).convert("RGB")
        mask = Image.open(mask_path)  # PNG indexé conseillé
        img_t = to_tensor(img)
        mask_t = torch.from_numpy(np.array(mask)).long()
        if self.transform:
            img_t = self.transform(img_t)
        return img_t, mask_t

# Transforms simples (à enrichir selon les besoins)
train_transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.RandomResizedCrop(256, scale=(0.8, 1.0)),
])
val_transform = None

# Dossiers à adapter
train_images = "data/train/images"
train_masks  = "data/train/masks"
val_images   = "data/val/images"
val_masks    = "data/val/masks"

train_set = SegmentationDataset(train_images, train_masks, train_transform)
val_set   = SegmentationDataset(val_images,   val_masks,   val_transform)

train_loader = DataLoader(train_set, batch_size=4, shuffle=True, num_workers=2, pin_memory=True)
val_loader   = DataLoader(val_set,   batch_size=4, shuffle=False, num_workers=2, pin_memory=True)

len(train_set), len(val_set)

## Modèle, loss et optimiseur

In [ ]:
# Nombre de classes (adapter à votre ontologie)
num_classes = 5

# Modèle DeepLabV3+ avec backbone ResNet-50
model = deeplabv3_resnet50(weights="DEFAULT")
model.classifier[-1] = nn.Conv2d(256, num_classes, kernel_size=1)
model = model.to(device)

# Loss cross-entropy (Dice peut être ajouté si besoin)
criterion = nn.CrossEntropyLoss()

# Optimiseur AdamW (LR et WD de départ)
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=1e-4)

## Métriques: IoU par classe et mIoU

In [ ]:
def compute_iou_per_class(pred, target, num_classes):
    # pred: logits [N,C,H,W] ou classes [N,H,W]; target: [N,H,W]
    if pred.ndim == 4:
        pred = pred.argmax(1)
    ious = []
    for cls in range(num_classes):
        pred_mask = (pred == cls)
        tgt_mask  = (target == cls)
        inter = (pred_mask & tgt_mask).sum().item()
        union = (pred_mask | tgt_mask).sum().item()
        if union == 0:
            ious.append(float("nan"))
        else:
            ious.append(inter / union)
    return ious, np.nanmean(ious)

## Boucles d'entraînement et validation

In [ ]:
def train_one_epoch(model, loader, optimizer, criterion):
    # Entraîne le modèle sur un epoch et retourne la loss moyenne
    model.train()
    total_loss = 0.0
    for imgs, masks in tqdm(loader, leave=False):
        imgs, masks = imgs.to(device), masks.to(device)
        logits = model(imgs)['out']
        loss = criterion(logits, masks)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / max(1, len(loader))

@torch.no_grad()
def validate(model, loader, criterion, num_classes):
    # Évalue le modèle, retourne loss, mIoU et IoU par classe moyens
    model.eval()
    total_loss = 0.0
    total_ious = []
    for imgs, masks in loader:
        imgs, masks = imgs.to(device), masks.to(device)
        logits = model(imgs)['out']
        loss = criterion(logits, masks)
        total_loss += loss.item()
        ious, miou = compute_iou_per_class(logits, masks, num_classes)
        total_ious.append(ious)
    # Moyenne des IoU par classe sur l'ensemble de la val
    if total_ious:
        per_class = np.nanmean(np.array(total_ious), axis=0)
        miou = np.nanmean(per_class)
    else:
        per_class = [float("nan")] * num_classes
        miou = float("nan")
    return total_loss / max(1, len(loader)), miou, per_class

## Entraînement complet avec MLflow

In [ ]:
num_epochs = 10  # ajustez
save_best_path = "best_model_deeplabv3.pth"
best_metric = -1.0

with mlflow.start_run(run_name="deeplabv3_baseline"):
    # Log paramètres principaux
    mlflow.log_param("model", "DeepLabV3+_ResNet50")
    mlflow.log_param("num_classes", num_classes)
    mlflow.log_param("epochs", num_epochs)
    mlflow.log_param("batch_size", train_loader.batch_size)
    mlflow.log_param("optimizer", "AdamW")
    mlflow.log_param("lr", 1e-4)
    mlflow.log_param("weight_decay", 1e-4)

    for epoch in range(num_epochs):
        t0 = time.time()
        train_loss = train_one_epoch(model, train_loader, optimizer, criterion)
        val_loss, val_miou, per_class_iou = validate(model, val_loader, criterion, num_classes)
        dt = time.time() - t0

        mlflow.log_metric("train_loss", train_loss, step=epoch)
        mlflow.log_metric("val_loss", val_loss, step=epoch)
        mlflow.log_metric("val_mIoU", float(val_miou), step=epoch)
        mlflow.log_metric("epoch_time_sec", dt, step=epoch)

        # Log IoU par classe
        for c, iou in enumerate(per_class_iou):
            if not np.isnan(iou):
                mlflow.log_metric(f"IoU_class_{c}", float(iou), step=epoch)

        print(f"[{epoch:02d}] train {train_loss:.4f} | val {val_loss:.4f} | mIoU {val_miou:.4f} | {dt:.1f}s")

        # Sauvegarde du meilleur modèle sur mIoU
        if val_miou > best_metric:
            best_metric = val_miou
            torch.save(model.state_dict(), save_best_path)
            mlflow.log_artifact(save_best_path)

    # Log du modèle final
    mlflow.pytorch.log_model(model, artifact_path="model")
    print("Entraînement terminé. Best mIoU:", best_metric)

## Visualisation qualitative

In [ ]:
# Sélection de quelques images du loader de validation pour visualiser
model.eval()
samples = 2
count = 0
for imgs, masks in val_loader:
    imgs = imgs.to(device)
    with torch.no_grad():
        logits = model(imgs)['out']
        preds = logits.argmax(1).cpu()
    for i in range(min(samples, imgs.size(0))):
        plt.figure(figsize=(10,3))
        show_triplet(imgs[i].cpu(), preds[i], title="Validation sample")
        plt.show()
    break